<a href="https://colab.research.google.com/github/JyotiChowrasia05/PySpark_Steephan/blob/main/PySpark_Steephan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install PySpark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import trim, col

In [ ]:
# Creating a Spark Session
spark = SparkSession.builder \
    .appName("MeteoriteApp") \
    .getOrCreate()

print("Spark Session Created Successfully!")

In [ ]:
# Reading the CSV file
csv_file_path = "/content/meteorite_landings_raw.csv"

df = spark.read.csv(
    csv_file_path,
    header=True,
    inferSchema=True
)

print(f"\nDataFrame created from '{csv_file_path}'")

In [ ]:
type(df)

In [ ]:
df.show()

In [ ]:
df.describe().show()

In [ ]:
df.printSchema()

In [ ]:
# Triming the whitr space from name column


In [ ]:
# Stop the SparkSession
spark.stop()
print("\nSparkSession stopped.")